In [1]:
## Looks like Churchland lab has analyzed this data on cloud. Let's see if it has the outputs we want
import fastplotlib as fpl
import numpy as np
import os
from typing import *
import tifffile
from one.api import ONE
one = ONE()
assert not one.offline, 'ONE must be connect to Alyx for searching imaging sessions'
from wfield import *
import wfield.cli as wfield_cli
import pathlib
from pathlib import Path
%load_ext autoreload

Unable to find extension: VK_EXT_acquire_drm_display
Unable to find extension: VK_EXT_physical_device_drm
No windowing system present. Using surfaceless platform
No config found!
No config found!
Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Valid,Device,Type,Backend,Driver
✅ (default),NVIDIA TITAN RTX,DiscreteGPU,Vulkan,555.42.02
❗ limited,"llvmpipe (LLVM 12.0.0, 256 bits)",CPU,Vulkan,Mesa 21.2.6 (LLVM 12.0.0)
❌,NVIDIA TITAN RTX/PCIe/SSE2,Unknown,OpenGL,3.3.0 NVIDIA 555.42.02


Max vertex attribute stride unknown. Assuming it is 2048
Max vertex attribute stride unknown. Assuming it is 2048


In [10]:
"""
The two datasets we will process are: 
(1) d34a502f-bd06-471f-8334-df41f785e1d9 (CSK-im-011)
(2) 3158300f-e72c-42fc-ac6c-c981615fe00f (CSK-im-001)

"""

'\nThe two datasets we will process are: \n(1) d34a502f-bd06-471f-8334-df41f785e1d9 (CSK-im-011)\n(2) 3158300f-e72c-42fc-ac6c-c981615fe00f (CSK-im-001)\n\n'

In [11]:
eid_used = '3158300f-e72c-42fc-ac6c-c981615fe00f'
one.list_datasets(eid_used)

['_ibl_experiment.description.yaml',
 'alf/#2025-07-29#/_ibl_trials.goCueTrigger_times.npy',
 'alf/#2025-07-29#/_ibl_trials.quiescencePeriod.npy',
 'alf/#2025-07-29#/_ibl_trials.stimOff_times.npy',
 'alf/#2025-07-29#/_ibl_trials.table.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_rightCamera.times.npy',
 'alf/_ibl_trials.goCueTrigger_times.npy',
 'alf/_ibl_trials.quiescencePeriod.npy',
 'alf/_ibl_trials.stimOff_times.npy',
 'alf/_ibl_trials.table.pqt',
 'alf/_ibl_wheel.position.npy',
 'alf/_ibl_wheel.timestamps.npy',
 'alf/_ibl_wheelMoves.intervals.npy',
 'alf/_ibl_wheelMoves.peakAmplitude.npy',
 'alf/widefield/imaging.imagingLightSource.npy',
 'alf/widefield/imaging.times.npy',
 'alf/widefield/imagingLightSource.properties.htsv',
 'alf/widefield/widefieldChannels.frameAverage.npy',
 'alf/widefield/widefieldLandmarks.dorsalCortex.json',
 'alf/widefield/widefieldSVT.haemoCorrected.npy',
 'alf/widefield/widefieldSVT.uncorrected.npy',
 'alf/widefield

In [69]:
collection = "raw_widefield_data"
raw_data_output = one.load_dataset(eid_used, 'imaging.frames.mov', collection=collection, download_only=True)

collection = "alf/widefield"

#Shape (2, height, width)
frame_average = one.load_dataset(eid_used, 'widefieldChannels.frameAverage.npy', collection=collection) 

#Shape (height, width, 200)
wf_U = one.load_dataset(eid_used, 'widefieldU.images.npy', collection=collection)

#Shape (200, 100000)
wf_SVT = one.load_dataset(eid_used, 'widefieldSVT.uncorrected.npy', collection=collection)[:, :100000]

#Shape (200, 10000)
wf_SVT_Hemo = one.load_dataset(eid_used, 'widefieldSVT.haemoCorrected.npy', collection=collection)[:, :100000]



# Run the widefield analysis on the downloaded data

In [2]:
raw_data_output = pathlib.Path("/data/lab/IBL_Alyx/churchlandlab_ucla/Subjects/FD_24/2023-06-07/001/raw_widefield_data/imaging.frames.mov")

In [68]:
# Run motion correction: 
wfield_cli._motion(raw_data_output.parent, nchannels=2)

In [67]:
wfield_cli._baseline(raw_data_output.parent, 30, nchannels=2)

In [ ]:
## Code to define a manual mask is to be run here

In [7]:
wfield_cli._decompose(raw_data_output.parent, k=200, nchannels=2)

In [6]:
wfield_cli._hemocorrect(raw_data_output.parent, fs=30, functional_channel=0, plot_ext='.png')

# Here is where we define the mask

In [2]:
raw_data_output = pathlib.Path("/data/lab/IBL_Alyx/zadorlab/Subjects/CSK-im-001/2021-01-19/001/raw_widefield_data/imaging.frames.mov")
dat = load_stack(raw_data_output.parent, nchannels=2)
std_img_gcamp = get_std_mask(dat[:, 0], threshold=64)
std_img_blood = get_std_mask(dat[:, 1], threshold=62)

In [3]:
updated_mask = np.logical_or(std_img_gcamp.astype('bool'), std_img_blood.astype('bool')).astype("float32")

In [5]:
iw = fpl.ImageWidget(data = [np.asarray(dat[:, 0]), std_img_gcamp, std_img_blood, updated_mask],
                    figure_shape = (1, 4))
iw.cmap = "gray"
iw.show()

In [6]:
save_path = raw_data_output.parent / "manual_mask.npy"
np.save(str(save_path), updated_mask)